In [1]:
#ディレクトリを中身ごと削除: shutil.rmtree() https://note.nkmk.me/python-os-remove-rmdir-removedirs-shutil-rmtree/
import datetime
now = datetime.date.today()
import shutil
#例外処理 https://note.nkmk.me/python-os-remove-rmdir-removedirs-shutil-rmtree/
try:
    shutil.rmtree('./trf/'+str(now)) 
    shutil.rmtree('./results/'+str(now)) 
    shutil.rmtree('./csv/'+str(now))
except OSError as e:
    pass

#Pythonで新しいディレクトリにファイルを作成・保存 https://note.nkmk.me/python-save-file-at-new-dir/
import os
os.mkdir('./trf/'+str(now)) 
os.mkdir('./results/'+str(now)) 
os.mkdir('./csv/'+str(now)) 

In [2]:
#https://shotanuki.com/python%E3%81%A7pdf%E3%82%92%E7%94%BB%E5%83%8F%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B/
#PythonでPDFを画像に変換する
# pdf2image 2回PIPでインストールできた
from pdf2image import convert_from_path
import glob

path_list = glob.glob('./mergedTRF/'+str(now)+'/*.pdf')
print(path_list)

['./mergedTRF/2019-07-02/31.pdf', './mergedTRF/2019-07-02/41-44.pdf', './mergedTRF/2019-07-02/21.pdf']


In [3]:
#PythonでOCRがんばる(1) hhttp://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916040943/
#tesseractでOCR〜数字だけでも〜 http://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916052946/
# p87 17.3.2 ステップ2：全ファイルをループして画像を開く https://www.oreilly.co.jp/pub/9784873117782/auto-sample.pdf

from PIL import Image
import cv2
import sys,os
import pyocr
import pyocr.builders
import csv
import datetime
now = datetime.date.today()

# nowフォルダのPDFをループする
path = './mergedTRF/'+ str(now)
for foldername in os.listdir(path):# ❶
    if os.path.isfile(os.path.join(path, foldername)):
        subfolder = os.path.basename(path)
        os.mkdir('./trf/'+str(now)+'/'+ foldername) 
        os.mkdir('./results/'+str(now)+'/'+ foldername) 

        images = convert_from_path(path+'/'+foldername)
        i = 0
        for image in images:
            image.save('./trf/' + str(now)+'/'+foldername+'/'+'trf_'+ str(now)+'_{}.png'.format(i), 'png')
            i += 1

        # カレントディレクトリの全画像をループする
        for filename in os.listdir('./trf/'+str(now)+'/'+foldername):# ❶
            ext = os.path.splitext(filename)
            if ext == '.png' or u'.jpeg' or u'.jpg':
                img = Image.open('./trf/'+str(now) +'/'+foldername+'/'+ filename)  # ❹
                img = img.crop((800,129,1050,190))
                img.save('./results/'+str(now) + '/'+foldername+"/PtID_" + str(filename))

                tools = pyocr.get_available_tools()
                if len(tools) == 0:
                    print("No OCR tool found")
                    sys.exit(1)
                # The tools are returned in the recommended order of usage
                tool = tools[0] 
                txt = tool.image_to_string(
                    Image.open('./results/' + str(now)+ '/'+foldername+ "/PtID_" + str(filename)),
                    lang="eng",
                    builder=pyocr.tesseract.DigitBuilder(tesseract_layout=6),
                    )
                path_w = './csv/'+ str(now)+'/'+str(now)+'.csv'
                with open(path_w, mode='a') as f:
                    f.write(txt+'\t')
                    f.write(filename+'\t')
                    f.write(foldername+'\n')
            
with open(path_w) as f:
     print(f.read())
    # New file


2003957	trf_2019-07-02_8.png	31.pdf
2350056	trf_2019-07-02_4.png	31.pdf
2347849	trf_2019-07-02_2.png	31.pdf
2349618	trf_2019-07-02_1.png	31.pdf
2003602	trf_2019-07-02_7.png	31.pdf
2348044	trf_2019-07-02_0.png	31.pdf
2735491 .	trf_2019-07-02_6.png	31.pdf
2345551	trf_2019-07-02_5.png	31.pdf
2347869	trf_2019-07-02_3.png	31.pdf
2710437	trf_2019-07-02_9.png	31.pdf
2137241	trf_2019-07-02_2.png	41-44.pdf
2737237	trf_2019-07-02_1.png	41-44.pdf
2692497	trf_2019-07-02_0.png	41-44.pdf
2737242	trf_2019-07-02_3.png	41-44.pdf
93019RE	trf_2019-07-02_8.png	21.pdf
2350832N	trf_2019-07-02_4.png	21.pdf
2040454	trf_2019-07-02_2.png	21.pdf
2251244	trf_2019-07-02_1.png	21.pdf
2280NK1T 7	trf_2019-07-02_7.png	21.pdf
2333931	trf_2019-07-02_0.png	21.pdf
22BNKR2E	trf_2019-07-02_6.png	21.pdf
2337179	trf_2019-07-02_5.png	21.pdf
27041 2E€Q	trf_2019-07-02_3.png	21.pdf
2327732	trf_2019-07-02_9.png	21.pdf

